In [11]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path
import os

# openai.api_key = os.getenv("OPENAI_API_KEY")

load_dotenv() # loads from .env file in project root
client = OpenAI() # uses my OPENAI_API_KEY env variable automatically

In [12]:
# Load CSVs
lyrics_df = pd.read_csv(Path("../data/lyrics.csv"))
comments_df = pd.read_csv(Path("../data/fan_comments.csv"))

lyrics_df.head(), comments_df.head()

(                 album        track                             lyric_excerpt  \
 0                Wings       Stigma          I’m sorry, I’m sorry, my brother   
 1   Map of the Soul: 7   Black Swan  Do your thing, do your thing with me now   
 2                   BE  Blue & Grey     Where is my angel? The end of the day   
 3                Proof  Yet To Come                      The best yet to come   
 4  Love Yourself: Tear    Fake Love             I'm so sick of this fake love   
 
     timestamp  
 0  2016-10-10  
 1  2020-02-21  
 2  2020-11-20  
 3  2022-06-10  
 4  2018-05-18  ,
          track                                        fan_comment   timestamp  \
 0  Blue & Grey        This song helped me through some dark days.  2021-03-01   
 1       Stigma                 The raw emotion here is unmatched.  2020-12-15   
 2  Yet To Come               I feel so hopeful when I hear this 💜  2022-06-12   
 3   Black Swan  Listening to this makes me realize my passion ...  2020-03

In [18]:
# GPT helper functions
def summarize_lyric(lyric):
    prompt = (
        f"Analyze the emotional themes of this BTS lyric:\n\n\"{lyric}\"\n\n"
        "Return three keywords and a 1-sentence summary of the emotion."
    )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def classify_comment_emotion(comment):
    prompt = (
        f"Classify this fan comment into one emotion: joy, sadness, nostalgia, inspiration.\n\n"
        f"Comment: \"{comment}\"\n\n"
        "Return the emotion and one sentence explaining why."
    )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    return response.choices[0].message.content.strip()

In [22]:
# Apply GPT to 3 rows only for demo (to limit token use)
# Updated to 6 rows after testing
lyrics_df_sample = lyrics_df.head(6).copy()
comments_df_sample = comments_df.head(6).copy()

lyrics_df_sample["gpt_summary"] = lyrics_df_sample["lyric_excerpt"].apply(summarize_lyric)
comments_df_sample["gpt_sentiment"] = comments_df_sample["fan_comment"].apply(classify_comment_emotion)

In [23]:
# Preview results
display(lyrics_df_sample[["track", "lyric_excerpt", "gpt_summary"]])
display(comments_df_sample[["track", "fan_comment", "gpt_sentiment"]])

,track,lyric_excerpt,gpt_summary
0,Stigma,"I’m sorry, I’m sorry, my brother","Keywords: Apology, Regret, Fraternal bond\nSum..."
1,Black Swan,"Do your thing, do your thing with me now","Keywords: encouragement, unity, freedom\nSumma..."
2,Blue & Grey,Where is my angel? The end of the day,"Keywords: longing, uncertainty, loneliness\nSu..."
3,Yet To Come,The best yet to come,"Keywords: Hope, Optimism, Anticipation\nSummar..."
4,Fake Love,I'm so sick of this fake love,"Keywords: Sick, Fake Love, Exhaustion\n\nSumma..."
5,Dynamite,Cause I-I-I'm in the stars tonight,"Keywords: wonder, excitement, freedom\nSummary..."


,track,fan_comment,gpt_sentiment
0,Blue & Grey,This song helped me through some dark days.,Sadness - The comment indicates that the perso...
1,Stigma,The raw emotion here is unmatched.,Inspiration - The comment suggests that the co...
2,Yet To Come,I feel so hopeful when I hear this 💜,Inspiration - The comment expresses a feeling ...
3,Black Swan,Listening to this makes me realize my passion ...,Inspiration - The comment expresses a renewed ...
4,Fake Love,Lyrics hit hard when you think about self-worth.,Inspiration - The comment suggests that the ly...
5,Dynamite,This song is pure serotonin boost!,"Joy - The use of the term ""serotonin boost"" in..."


In [24]:
# Save results
lyrics_df_sample.to_csv("../data/lyrics_gpt_enriched.csv", index=False)
comments_df_sample.to_csv("../data/fan_comments_gpt_enriched.csv", index=False)